<h2> More on Tensorflow </h2> 

GPUS dramatically speed up computations by splitting computations into many smaller chunks and running them in parallel across many GPU threads. 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [3]:
a.dtype

dtype('int32')

In [4]:
b = tf.Variable(a)
b

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [5]:
b.scatter_nd_update(indices = [[1,1]],updates = [100])
b

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[  1,   2,   3],
       [  4, 100,   6]])>

In [6]:
#creating a simple tensor
t = tf.constant([[1,2,3],[4,5,6]])
t.shape

TensorShape([2, 3])

In [7]:
#indexing
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [5, 6]])>

In [8]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]])>

In [9]:
tf.matmul(t,tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

<h3> Tensors and Numpy </h3> 

In [10]:
import numpy as np
a = np.array([2,4,5])

In [11]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [12]:
#casting from 32bit float to a 64 bit float to sum in TF
t2 = tf.constant(40.,dtype = tf.float64)
t2

<tf.Tensor: shape=(), dtype=float64, numpy=40.0>

In [13]:
tf.constant(2.0) + tf.cast(t2,tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

<h3> Variables in Tensorflow </h3> 

We cannot modify constant tensors, however we can modify tf.Variable. These are required as we need to tweak weights in a neural network. 

In [14]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

A tf.Variable acts like a constant tensor and allows to perform the same operations. But it also allows you to modify th variable using the assign() method.

In [15]:
a = tf.constant([[1,2,3],[4,5,6]])
a

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [16]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [17]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [18]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

In [19]:
v.scatter_nd_add(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[200.,  42.,   6.],
       [  8.,  10., 400.]], dtype=float32)>

scatter methods allows you to specify the index position and add to it

<h3> Sparse Tensors </h3> 

In a Sparse Tensor, you always need to give the positions first, then the values, then the dense shape

In [20]:
s = tf.SparseTensor(indices = [[0,1],[1,0],[2,3]],
                    values = [1.,2.,3.], 
                    dense_shape=[3,4])

In [21]:
tf.sparse.to_dense(s)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

Sparse tensors need to always be given in order of indices

In [22]:
s5 = tf.SparseTensor(indices=[[0, 1], [0, 2]],
                     values=[1., 2.],
                     dense_shape=[3, 4])

In [23]:
tf.sparse.to_dense(s5)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 2., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

<h3> Tensor Arrays </h3>

We first crrate the tensor with a fixed size. we cannot add past this

In [24]:
array = tf.TensorArray(dtype = tf.float32,size = 3)
array = array.write(0,tf.constant([1.,2.]))
array = array.write(1,tf.constant([3.,10.]))
array = array.write(2,tf.constant([5.,7.]))

<h3> Customizing models and training algorithms </h3> 

<h3> Custom Loss Functions </h3> 

In [25]:
#where y_true - y_pred absolute is less than 1, we want the squared loss and linear loss
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error,squared_loss,linear_loss)

#we need to always use tensorflow functions to be used as custom loss functions. 

For the next step, we just apply this to our keras model

model.compile(loss = huber_fn, optimizer = "adam")

model.fit(.....)

In [26]:
#current implementation allows us to have one threshold of 1 in the Huber Function
#How to change the threshold?
def create_huber(threshold = 1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_errors = tf.abs(error) <  threshold
        #mae loss
        squared_loss = tf.square(error)/2
        #mse loss
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        #if your error is less than threshold, return squared loss, 
        #else return linear_loss
        return tf.where(is_small_errors, squared_loss,linear_loss)
    return huber_fn

model.compile(loss = create_huber(2.0), optimizer = "adam")

The above causes an issue when saving the model using 
keras.callbacks.ModelCheckpoint("modelname.h5", save_best_only = True)

When we load the model again we will have to specify which function is being called for the loss function as the loss function is a custom one here

model = keras.models.load_model("modelname.h5", custom_objects = {"huber_fn":create_huber(2.0)})

In [27]:
from tensorflow import keras

In [28]:
#to avoid the above we can inherit the Keras.losses.loss method

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        #instantiate the superclass
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        #you get the base configuration of superclass
        base_config = super().get_config()
        #add in the threshold and creates a new dictionary to be returned
        return {**base_config, "threshold":self.threshold}

get_config method returns a dictionary mapping each hyperparameter name to its value. It first calls the parent class's get_config() method, then adds the new hyperparameters to this dictionary.

from the above, when loading a model, we do this:

**model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
custom_objects={"HuberLoss": HuberLoss})**

Now we dont have to provide the threshold value too.

When we save the mode, Keras calls the loss instance (HuberLoss) get_config() method and the returned dictionary is stored as a JSON in the h5 file. 

When loaded, it calls **from_config()** and creates the instance of the class, passing the return from **from_config()** to **kwargs.

<h3>Custom Activation Functions, Initializers, Regularizers, and Constraints</h3>

In [29]:
#defining custom activation functions 
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

In [30]:
def my_glorot_initializer(shape,dtype = tf.float32):
    stddev = tf.sqrt(2/(shape[0] + shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

In [31]:
def my_li_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [32]:
#function to return only positive weights
def my_positive_weights(weights):
    return tf.where(weights<0, tf.zeros_like(weights),weights)

In [33]:
#custom regularizer using subclassing

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,regfactor):
        self.factor = regfactor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"regfactor":self.factor}

For losses, layers and models we implement the call() method, and for regularizers, initializers and constraints we use the _____call_____() method. 

<h3>Custom Metrics </h3> 



In [34]:
a = tf.Variable([[2,2,2],[2,2,2]])
a = tf.cast(a,tf.float32)
a

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>

In [35]:
tf.reduce_sum(tf.abs(0.01*a))

<tf.Tensor: shape=(), dtype=float32, numpy=0.11999999>

At each training step the weights will be passed to the regularization function to compute the regularization loss. The return is then added to the main loss to get the final loss used for training. 

<h3> Custom Metrics </h3> 

Usually metrics keep track of the mean of a metric from each epoch. 

Suppose we have 5 true predictions, but 4 true positives. Thats 0.8 precision. Next epoch, we have 3 true predictions with 0 true positives. Thats 0 precision, but if we use mean, it becomes 0.4 precision overall

But the actual one is 8 true predictions with 4 true positives in total. Thats 0.5 precision. Hence we need an object to track the true positives and an object to track the predictions. 


In [36]:
from tensorflow import keras
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [37]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

Note above that the same precision object, under 2 runs prduces the overal precision of the data fed into it. Not a mean, or not a new precision for the the new run.

This is called a streaming metric. 

In [38]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [39]:
def create_huber(threshold = 1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_errors = tf.abs(error) <  threshold
        #mae loss
        squared_loss = tf.square(error)/2
        #mse loss
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        #if your error is less than threshold, return squared loss, 
        #else return linear_loss
        return tf.where(is_small_errors, squared_loss,linear_loss)
    return huber_fn

In [166]:
#creating streaming metrics
class HuberMetric(keras.metrics.Metric):
    def __init__(self,threshold = 1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        #returns a huber function to be used in this class
        self.huber_fn = create_huber(threshold)
        #add weight creates variables to keep track of attributes
        self.total = self.add_weight("total",initializer= "zeros")
        self.count = self.add_weight("count",initializer= "zeros")
    def update_state(self,y_true,y_pred,sample_weight = None):
        result = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(result))
        self.count.assign_add(tf.cast(tf.size(y_pred), tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config = self.get_config()
        return {**base_config, "threshold":self.threshold}

In [181]:
class customPrecision(keras.metrics.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def update_state(self,y_true, y_pred,sample_weight = None):
        try: 
            for _class in self.unique_classes:
                self.total_true[_class] = self.total_true[_class] + tf.where(ytrue == _class).shape[0]
                self.total_prediction[_class] = self.total_prediction[_class] + tf.where(ypred == _class).shape[0]
                self.total_truepositives[_class] = self.total_truepositives[_class] + len([i for i in tf.where(ypred == _class) if i in tf.where(ytrue == _class)])
            #return [self.total_prediction,self.total_truepositives]
        except:
            self.unique,_,_ = tf.unique_with_counts(ytrue)
            #obtains the unique classes to a list that we can loop through
            self.unique_classes = self.unique.numpy()
            self.total_truepositives = {_class:0 for _class in self.unique_classes}
            self.total_prediction = {_class:0 for _class in self.unique_classes}
            self.precision_classbase = {_class:0 for _class in self.unique_classes}
            self.total_true = {_class:0 for _class in self.unique_classes}
            for _class in self.unique_classes:
                self.total_true[_class] = self.total_true[_class] + tf.where(ytrue == _class).shape[0]
                self.total_prediction[_class] = self.total_prediction[_class] + tf.where(ypred == _class).shape[0]
                self.total_truepositives[_class] = self.total_truepositives[_class] + len([i for i in tf.where(ypred == _class) if i in tf.where(ytrue == _class)])
            #return [self.total_prediction,self.total_truepositives]
    def result(self):
        for _class in self.unique_classes:
            self.precision_classbase[_class] = self.total_truepositives[_class]/self.total_prediction[_class]
            self.precision_classbase[_class] = self.precision_classbase[_class] * (self.total_true[_class]/sum(self.total_true.values()))
        return sum(self.precision_classbase.values())
    def get_config(self):
        base_config = self.get_config()
        return {**base_config, "threshold":self.threshold}       

In [182]:
ytrue = tf.Variable([1,0,1,1,0,1,1,0,1])
ypred = tf.Variable([1,1,0,0,1,1,1,0,0])

In [183]:
prec_test = customPrecision()
prec_test(ytrue,ypred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.48333332>

In [163]:
res

In [164]:
prec_test.precision_classbase

{1: 0, 0: 0}

In [170]:
hub = HuberMetric()
hub(ytrue,ypred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.2777778>

In [168]:
ytrue

<tf.Variable 'Variable:0' shape=(9,) dtype=int32, numpy=array([1, 0, 1, 1, 0, 1, 1, 0, 1])>

In [176]:
a = {0:4,1:3}

In [177]:
a

{0: 4, 1: 3}

In [179]:
sum(a.values())

7